In [ ]:
# Inserting Multicategories into Reviews
> "Produce Latent Factors, select the higher weights and use the ordinality to create multi-categories assignments to Movie Reviews. Finally re-classify the reviews per film and identify dominance."

- toc:true
- branch: master
- badges: true
- comments: true
- author: Conwyn
- categories: [fastai,language,ordinality]

In [ ]:
%matplotlib inline
%reload_ext autoreload
 
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
from fastai import *
!pip install git+https://github.com/alberanid/imdbpy


     |████████████████████████████████| 727kB 4.3MB/s 
     |████████████████████████████████| 51kB 7.4MB/s 
     |████████████████████████████████| 204kB 40.0MB/s 
     |████████████████████████████████| 1.2MB 38.2MB/s 
     |████████████████████████████████| 51kB 7.9MB/s 
     |████████████████████████████████| 61kB 8.3MB/s 
Mounted at /content/gdrive
  Cloning https://github.com/alberanid/imdbpy to /tmp/pip-req-build-hmq9dz76
  Running command git clone -q https://github.com/alberanid/imdbpy /tmp/pip-req-build-hmq9dz76
  Created wheel for IMDbPY: filename=IMDbPY-2021.5.12-cp37-none-any.whl size=299286 sha256=d58b37342e012a2b2db7958a97fe1fb458967157d5017d9fad86eef58458ebc9
  Stored in directory: /tmp/pip-ephem-wheel-cache-63933vf_/wheels/0f/09/61/190df5e0276765680540f1562f2abca80e725a7e48595e993f
Successfully built IMDbPY


Define the number of higher weights to investigate (FACTORS) and the number of heaviest films (Twenty).

In [ ]:
from fastai.collab import *
from fastai.tabular.all import *
FACTORS=5             #Take the 10 latent factors
Twenty = 100           #Only analyse 100 films
# We need to bring in a prepared WiKi IMDB language model encoder and its vocab.
import pickle
import collections

from imdb import IMDb



COLAB has no data retained between sessions so data is stored on your Google drive. The encoder is expected to be found in a folder called models

In [ ]:
#import Wiki/IMDB language encoder.

!mkdir /content/models


In [ ]:
#finetunedE
!cp /content/gdrive/MyDrive/finetunedE.pth /content/models

In [ ]:
#replicate wiki/IMDB vocab
!cp  /content/gdrive/MyDrive/savelm.p /content

dls_lm = pickle.load( open( "/content/savelm.p", "rb" ) )

From the book. Get Movie Lens and identify 50 Latent Factors

In [ ]:
# Get Movie Lens user/movie ratings and find the latent factors


path = untar_data(URLs.ML_100k)

path.ls()


ratings = pd.read_csv(path/'u.data',delimiter='\t',header=None,names=['user','movie','rating','timestamp'])
ratings.head()
movies = pd.read_csv(path/'u.item',delimiter='|',encoding='latin-1',usecols=(0,1),names=('movie','title'),header=None)
movies.head()
ratings=ratings.merge(movies)
ratings.head()



dls = CollabDataLoaders.from_df(ratings,item_name='title',bs=64)
dls.show_batch()


,user,title,rating
0,542,My Left Foot (1989),4
1,422,Event Horizon (1997),3
2,311,"African Queen, The (1951)",4
3,595,Face/Off (1997),4
4,617,Evil Dead II (1987),1
5,158,Jurassic Park (1993),5
6,836,Chasing Amy (1997),3
7,474,Emma (1996),3
8,466,Jackie Chan's First Strike (1996),3
9,554,Scream (1996),3


In [ ]:
learn = collab_learner(dls, n_factors=50, y_range = (0,5.5))
learn.fit_one_cycle(5,5e-3,wd=0.1)

epoch,train_loss,valid_loss,time
0,0.959085,0.955102,00:07
1,0.869222,0.876647,00:07
2,0.745436,0.841352,00:07
3,0.596926,0.825848,00:07
4,0.482670,0.826195,00:07


 Now look at the latent weights for each movie and pick the highest (FACTORS) weights  and then using the movie title find the IMDB reviews later

In [ ]:
#hide_output



movie_weight = learn.model.i_weight.weight

idxs = movie_weight.argsort(descending=True)

idxs5 = idxs[:,0:FACTORS]   # Originally 5 

movie_weight5 = [movie_weight[i,idxs5[i]] for i in range(movie_weight.shape[0])]

idxm5s = movie_weight[:,0].argsort(descending=True)

movie_weight5top = []
for i in range(FACTORS):    #idxm5s:
   movie_weight5top.append(movie_weight5[idxm5s[i]])

movie_weight5toptitles = []

for i in range(FACTORS):    #idxm5s:
   T0 = idxm5s[i].item()  
   T1 = movies.iloc[T0].title
   movie_weight5toptitles.append(T1)

newidxs=idxs




#now give the review the attribute;idxs contains the latency number we just need to idxm5scpu to pull the high line and generate Lx Ly Lz assoiated with text 

idxs5[0:FACTORS,0:FACTORS]

*** Fled (1996) *** 24 reviews
Fled is a cheerfully illogical actioner which rattles along at a seriously rapid pace without ever pausing for breath. It's stupid and spectacular in equal measure.The plot hardly seems important, but for what it's worth here it is: a Cuban mafia boss has foolishly allowed himself to leave some incriminating evidence on a computer disk. The only guy who knows where the whereabouts of the disk is in a chain gang in the Deep South. An undercover cop is sent to the chain gang to find out where the disk is, and to keep the guy alive when the Cuban hitmen turn up to silence him.The first half of the movie borrows liberally from The Defiant Ones, with Baldwin and Fishburne chained together and fleeing through the swamps from both the good guys and the bad guys. Later on, they reach the big city, and the film then borrows ideas from any police actioner that you could care to mention (Lethal Weapon, The Dead Pool, Tango and Cash, Striking Distance, et al). In fac

tensor([[20, 10,  7, 17, 22],
        [39, 32, 21, 29,  4],
        [12, 22, 24, 46, 41],
        [ 3, 15, 32, 41, 17],
        [20, 27,  7, 35, 34]], device='cuda:0')

In [ ]:
#hide
# for i in movie_weight5toptitles:
#   ia = IMDb()
#   imovies = ia.search_movie(i) 
#   mi = imovies[0].movieID
#   imdbdata = ia.get_movie_reviews(str(mi))['data'] # ['reviews']
#   if 'reviews' in imdbdata: 
#     details = imdbdata['reviews']
#     print(f'*** {i} *** {len(details)} reviews')
#     for j in details: 
#       print(j['content'])
#       print('')


In [ ]:
# Find the most common latent factors

idxs5list=[]
for iii in range(idxs5.shape[0]):
  for jjj in range(idxs5.shape[1]):
    idxs5list.append(idxs5[iii][jjj].item())
    
counter=collections.Counter(idxs5list)

print(counter.most_common(20))



[(3, 318), (15, 298), (12, 272), (42, 258), (37, 254), (13, 252), (31, 247), (22, 244), (10, 231), (4, 224), (41, 218), (2, 212), (0, 211), (29, 209), (27, 209), (39, 203), (7, 196), (34, 184), (35, 182), (16, 182)]


In [ ]:
Latency=idxs[idxm5s[:],0:FACTORS]

I notice that Latent Factor zero was dominating the analysis so here I remove it.

In [ ]:
# Now remove all movies with a latent factor of zero. 211 filmes. It is almost if latency zero = it is a film
def tensorvaluezero(x):
  sum = False
  for i in x:
    e = i.item()
    if (e == 0.0): sum=True
  return sum
TCList = []
for i,v in enumerate(Latency): 
  if (tensorvaluezero(v.cpu()) == False ) : 
    TCList.append(i)
    


In [ ]:
len(TCList),len(Latency)

-211

Just an idea but try find films with similar root mean square of ordinality.

In [ ]:
# #Try and find similar valued tensors warning very primative
# def tensorvalue(x):
#   sum = 0.
#   for i in x:
#     e = i.item()
#     sum =+ (e*e)
#   return math.sqrt(sum)
# TCList = []
# TC = tensorvalue(tensor([3,15,12,42,37]))

# for i,v in enumerate(Latency): 
#   if (tensorvalue(v.cpu()) == TC ) : 
#     TCList.append(i)
#     print(i)

80
97
113
147
171
197
330
375
386
553
583
595
693
746
759
769
806
851
902
903
968
1001
1073
1210
1222
1239
1352
1377
1399
1414
1512
1538
1584


In [ ]:
Latency

tensor([[12,  0, 15, 19,  9],
        [ 0,  3, 35,  4, 28],
        [ 0,  2, 28, 31,  9],
        ...,
        [21, 15,  3, 33, 39],
        [10,  5, 41,  6, 21],
        [49, 36, 39, 33, 43]], device='cuda:0')

In [ ]:
md=movies.iloc[idxm5s[:].cpu()].title


Choose the top "Twenty" movies with the highest weight for the first latent weight

In [ ]:
# Choose the top "Twenty" movies with the highest weight for the first latent weight

top20movies = md.to_list()[0:Twenty]

In [ ]:
# Warning 
print(TCList[0],movies.iloc[TCList[0]])
Twenty = len(TCList)
top20movies = [md.to_list()[i] for i  in TCList][0:20]


23 movie                            24
title    Rumble in the Bronx (1995)
Name: 23, dtype: object


In [ ]:
top20movies[0]

'Jade (1995)'

Here we take the Latent Factors for the Movies and replicate to the Movie reviews from IMBD

In [ ]:
LatencyString=[]
for j in Latency[0:Twenty]:
  LT =''
  temp_i = len(j)
  for k in j:
    MN = (k.item())
    temp_i -= 1
    if temp_i > 0 :
      LT=LT+str(k.item())+';'
    else: 
      LT=LT+str(k.item())
     
    
  LatencyString.append(LT)



In [ ]:
LatencyString[0:10]

['12;0;15;19;9',
 '0;3;35;4;28',
 '0;2;28;31;9',
 '0;8;28;13;45',
 '0;3;46;35;8',
 '0;3;15;39;36',
 '35;0;7;8;10',
 '31;15;0;40;13',
 '0;15;29;12;20',
 '0;9;8;45;14']

In [ ]:
#Now retrieve the reviews for these "twenty" movies. Movies may not have a review. 
pdload=[]
reviewCount = []
for i in range(len(top20movies)):
  ia = IMDb()
  imovies = ia.search_movie(str(top20movies[i])) 
  if imovies != [] :
    mi = imovies[0].movieID
    imdbdata = ia.get_movie_reviews(str(mi))['data'] # ['reviews']
    if 'reviews' in imdbdata: 
      details = imdbdata['reviews']
      print(f'*** {i} *** {len(details)} reviews')
      reviewCount.append((top20movies[i],len(details)))
      for j in details: 
        #print(j['content'])
       #print('')
        pdload.append([top20movies[i],LatencyString[i],j['content']])

*** 0 *** 24 reviews
*** 1 *** 2 reviews
*** 2 *** 11 reviews
*** 3 *** 25 reviews
*** 4 *** 19 reviews
*** 5 *** 25 reviews
*** 6 *** 25 reviews
*** 8 *** 24 reviews
*** 9 *** 24 reviews
*** 10 *** 25 reviews
*** 12 *** 25 reviews
*** 13 *** 25 reviews
*** 14 *** 25 reviews
*** 15 *** 15 reviews
*** 16 *** 21 reviews
*** 17 *** 25 reviews
*** 18 *** 25 reviews
*** 19 *** 25 reviews


In [ ]:
pdload[0]


['Jade (1995)',
 '12;0;15;19;9',
 'To say Friedkin\'s career has had its ups and downs is an understatement, his eighties filmography inarguably has enough bombs to sink a oil tanker. Yet eschewing their performances at the box office, many of his films yearn to be rediscovered, from "Cruising" to "Deal of the Century" to "Rampage". Let\'s not kid ourselves, "Jade" is not a great film, and this is the fault of one man and one man alone - Joe Esterhas. If trash had a messiah, it would be him. For a fleeting moment in the nineties, Esterhas was paid by the bucketload to write formulaic movies for guys, and the erotic thriller has him to thank for its continuing lugubrious existence. "Jade" is interesting however, it is an erotic thriller without the erotic part. While Paul Verhoeven filled "Basic Instinct" chock full of the sleaze he had become renowned for, Friedkin\'s films are notable for primarily dealing with male characters, and are subsequently about as erotic as as a bowl of cere

In [ ]:
dfr=pd.DataFrame(pdload,columns=['Movie','Latency','Review'])

In [ ]:
dfr.iloc[0]

Movie                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  Jade (1995)
Latency                                                                                                                                                                                                                                                                                                                                                                                              

Now create the classifier but use vocab=dls_lm.vocab which we loaded from the Wiki IMDB dataloader


In [ ]:
#Now create the classifier but use vocab=dls_lm.vocab
from fastai.text.all import *

dlsr = TextDataLoaders.from_df(df=dfr, text_vocab=dls_lm.vocab,text_col='Review', label_col='Latency', label_delim=";",y_block=MultiCategoryBlock,splitter=RandomSplitter(0.2) )
dlsr.show_batch(max_n=3)

dlsr.vocab[1]

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


,text,None
0,"xxbos xxmaj the xxmaj great xxmaj famous as well as the xxmaj great xxmaj infamous occurrences in a nation 's history are and always will be looked upon as subjects worthy of a film treatment . xxmaj whether an incident were to be filmed from a story and its screenplay form written directly for the screen or an adaptation of a xxmaj novel , a xxmaj short xxmaj story or a xxmaj stage xxmaj play , they are an important element of formulating our image of our xxmaj world . xxmaj right or wrong , it is a fact of life and one that we must learn to live xxunk good example of just what we are driving at is the occurrences of xxmaj october 26 , 1881 in xxmaj tombstone , xxmaj arizona . xxmaj known as "" the xxmaj gunfight at xxup o.k . xxmaj corral "" ,",0;16;27;34;9
1,"xxbos i often hear similar stories of people 's first experiences watching xxmaj blade xxmaj runner , finding the film dull but coming to appreciate it years later - my story has the same trajectory . i first tried to watch xxmaj blade xxmaj runner ( of what i believe was xxmaj the xxmaj final xxmaj cut ) on xxup tv in xxmaj christmas 2009 , only to stop watching after half an hour due to boredom . xxmaj over the years , however , i would be compelled to return to xxmaj blade xxmaj runner several times and get more out of it with each viewing . xxmaj the tech - noir world of xxmaj blade xxmaj runner is one to get lost in with its use of neon and many billboards of geishas , albeit a more depressing , dystopian one than say that of xxmaj star xxmaj",0;13;28;45;8
2,"xxbos xxmaj who could have possibly thought that xxmaj oliver xxmaj stone would surpass the level reached by "" jfk "" , one of the greatest political thrillers ever made ? xxmaj yet his "" nixon "" is not only the harrowing political biography of the most controversial and unpopular xxmaj president of the xxmaj united xxmaj states , but also the gripping psychological study of a tormented man who got too much to prove . xxmaj it 's also a terrific movie served by an impressive casting : xxmaj anthony xxmaj hopkins , xxmaj joan xxmaj allen , xxmaj james xxmaj woods , xxmaj bob xxmaj hoskins , xxmaj ed xxmaj harris , xxup e.g. xxmaj marshall , xxmaj madeline xxmaj kahn and an unrecognizable xxmaj paul xxmaj sorvino as xxmaj henry xxmaj kissinger . xxmaj and it 's definitely one of the best movies of 1995 , which",0;28;36;4;9


['0', '10', '11', '12', '13', '14', '15', '16', '17', '19', '2', '20', '24', '25', '27', '28', '29', '3', '31', '34', '35', '36', '38', '39', '4', '40', '42', '45', '46', '47', '49', '7', '8', '9']

Using our Wiki/IMDB language model encoder let us build a classifier in order to predict the latent factors from the reviews. So we are building a classifier which is looking for the number of latent factors which were present in the top "Twenty" films. We are using the pre-trained Wiki-IMDB encoder and the IMDB vocab used in the pre-training.


In [ ]:
#Using our Wiki/IMDB language model encoder let us build a classifier in order to predict the latent factors from the reviews 
learnr = text_classifier_learner(dlsr, AWD_LSTM, drop_mult=0.5, n_out=len(dlsr.vocab[1]), metrics=[]).to_fp16()




In [ ]:
# How many latent factors did we find. NB no 44 in the 100 films
(len(dlsr.vocab[1]),len(counter),len(dls_lm.vocab),len(dlsr.vocab[0]),len(dlsr.vocab[1]))
#dls_lm.vocab

(34, 50, 60008, 60008, 34)

In [ ]:
#Use the encoder from Wiki/IMDB
learnr.load_encoder('finetunedE')

In [ ]:
#ALL GOOD NB encoder needs to be in /content/models but saved to path/models

In [ ]:
learnr.fit_one_cycle(1,2e-2)


epoch,train_loss,valid_loss,time
0,0.713931,0.663639,00:02


Following the box and slowly unfreezing

In [ ]:
learn.freeze_to(-2)
learnr.fit_one_cycle(1,slice(1e-2/(2.6**4),1e-2))
learn.freeze_to(-3)
learnr.fit_one_cycle(1,slice(5e-3/(2.6**4),5e-3))
learnr.unfreeze()
learnr.fit_one_cycle(2,slice(1e-3/(2.6**4),1e-3))


epoch,train_loss,valid_loss,time
0,0.655238,0.648486,00:02


epoch,train_loss,valid_loss,time
0,0.632271,0.661978,00:02


epoch,train_loss,valid_loss,time
0,0.625277,0.677263,00:04
1,0.619720,0.683465,00:04


In [ ]:
dfr.iloc[0,2]

'To say Friedkin\'s career has had its ups and downs is an understatement, his eighties filmography inarguably has enough bombs to sink a oil tanker. Yet eschewing their performances at the box office, many of his films yearn to be rediscovered, from "Cruising" to "Deal of the Century" to "Rampage". Let\'s not kid ourselves, "Jade" is not a great film, and this is the fault of one man and one man alone - Joe Esterhas. If trash had a messiah, it would be him. For a fleeting moment in the nineties, Esterhas was paid by the bucketload to write formulaic movies for guys, and the erotic thriller has him to thank for its continuing lugubrious existence. "Jade" is interesting however, it is an erotic thriller without the erotic part. While Paul Verhoeven filled "Basic Instinct" chock full of the sleaze he had become renowned for, Friedkin\'s films are notable for primarily dealing with male characters, and are subsequently about as erotic as as a bowl of cereal. "Jade" is not about sex; it is

In [ ]:
learnr.predict(dfr.iloc[0,2])

((#13) ['0','12','14','15','16','17','19','2','27','31'...],
 tensor([ True, False, False,  True, False,  True,  True,  True,  True,  True,  True, False, False, False,  True, False, False, False,  True,  True, False, False, False, False, False,  True, False,
         False, False, False, False, False, False,  True]),
 tensor([0.5525, 0.4047, 0.4269, 0.6026, 0.4509, 0.5422, 0.6147, 0.5337, 0.6194, 0.5705, 0.5010, 0.4811, 0.4085, 0.4697, 0.5276, 0.4015, 0.4815, 0.4198, 0.5133, 0.5024, 0.4524, 0.4466, 0.4367, 0.4472,
         0.4352, 0.5325, 0.4537, 0.4863, 0.4394, 0.4571, 0.4811, 0.3731, 0.3053, 0.5401]))

Now predict the latent factors for each movie based on their reviews

In [ ]:
#Now predict the latent factors for each movie based on their reviews
superpredictions=[]
accRC=0
for m in range(len(reviewCount)):
  RC = reviewCount[m][1]
  Film = reviewCount[m][0]
  predictions=[]
  print(f'film  {Film} reviews {RC}')
  for l in range(reviewCount[m][1]+0):
    predictions.append(learnr.predict(dfr.iloc[accRC+l,2])[0])
  accRC=accRC+RC
  superpredictions.append((Film,predictions))

film  Jade (1995) reviews 24


film  Paris Was a Woman (1995) reviews 2


film  Princess Bride, The (1987) reviews 11


film  Blade Runner (1982) reviews 25


film  Letter From Death Row, A (1998) reviews 19


...

film  Naked in New York (1994) reviews 15


film  Bewegte Mann, Der (1994) reviews 21


film  If Lucy Fell (1996) reviews 25


film  Some Like It Hot (1959) reviews 25


film  Twilight (1998) reviews 25


In [ ]:
(superpredictions[0][0],superpredictions[0][1])

('Jade (1995)',
 [(#13) ['0','12','14','15','16','17','19','2','27','31'...],
  (#11) ['0','12','15','16','17','19','20','27','29','34'...],
  (#21) ['0','12','13','14','15','16','17','19','2','25'...],
  (#15) ['0','11','12','13','14','15','16','17','20','27'...],
  (#13) ['0','12','13','14','15','16','17','27','29','34'...],
  (#12) ['0','12','15','16','17','19','20','27','38','40'...],
  (#16) ['0','12','14','15','16','17','19','2','20','25'...],
  (#12) ['0','12','15','16','17','19','2','27','31','34'...],
  (#13) ['0','12','15','16','17','20','25','27','29','34'...],
  (#11) ['0','12','15','17','19','20','25','29','34','40'...],
  (#9) ['0','12','14','15','17','19','27','31','34'],
  (#21) ['0','12','13','14','15','16','17','2','20','25'...],
  (#20) ['0','11','12','13','16','17','2','24','27','28'...],
  (#10) ['0','12','13','15','16','17','19','27','29','34'],
  (#13) ['12','13','15','16','17','19','20','24','27','29'...],
  (#11) ['0','12','15','17','19','20','29','34','40','49

Now we are going to plot the predicted Latent Factors per movie

In [ ]:
#Prepare the matplot lib format
import math
int(math.sqrt(len(superpredictions)))

4

In [ ]:
superpredictions[4]

('Letter From Death Row, A (1998)',
 [(#22) ['0','11','12','13','16','17','20','24','27','28'...],
  (#20) ['0','11','12','13','14','15','16','17','24','27'...],
  (#17) ['11','13','16','17','24','25','27','35','38','39'...],
  (#15) ['0','11','13','14','16','17','24','27','3','34'...],
  (#19) ['0','11','13','14','16','17','2','24','27','28'...],
  (#20) ['11','13','16','17','24','25','27','28','29','34'...],
  (#21) ['0','12','13','14','15','16','17','2','25','27'...],
  (#18) ['0','13','14','15','16','17','2','27','28','3'...],
  (#21) ['0','11','13','16','17','20','24','25','27','3'...],
  (#20) ['0','11','13','16','17','2','20','28','29','3'...],
  (#19) ['11','13','16','17','20','24','25','27','34','35'...],
  (#19) ['0','11','13','15','16','17','24','25','27','3'...],
  (#18) ['11','16','20','24','25','27','28','34','35','36'...],
  (#22) ['0','11','13','16','17','2','24','25','27','28'...],
  (#22) ['11','12','13','15','16','17','2','24','25','27'...],
  (#20) ['0','13','16','1

In [ ]:
reviewCount[0:10]

[('Jade (1995)', 24),
 ('Paris Was a Woman (1995)', 2),
 ('Princess Bride, The (1987)', 11),
 ('Blade Runner (1982)', 25),
 ('Letter From Death Row, A (1998)', 19),
 ('Ice Storm, The (1997)', 25),
 ('Paradise Lost: The Child Murders at Robin Hood Hills (1996)', 25),
 ("Ulee's Gold (1997)", 24),
 ("C'est arrivé près de chez vous (1992)", 24),
 ('Pretty Woman (1990)', 25)]

From observation COLAB likes all the matplotlib in one cell

In [ ]:
#Build a 2D shape to hold all the histograms
matplotlib.rcParams['figure.figsize'] = [256, 64*1024/256-1]
import math
pict = int(math.sqrt(len(superpredictions)))+1
if pict > 8 : pict = 8
pictw = pict


picth = ((len(superpredictions))//pict) + 1
print(picth,pictw)
_,axs = plt.subplots(picth,pictw)
#axs.ravel()
for boxcount,sp in enumerate(superpredictions):
  #print(sp[0])
  reviewCountForTitle = reviewCount[boxcount][1]
  cord = (boxcount//(pictw),boxcount-(boxcount//pictw)*pictw)
  #cord = boxcount
  flat_list = [item for sublist in sp[1] for item in sublist]
  flat_list.sort()
  axs[cord].hist(flat_list, density=False, bins=len(dlsr.vocab[1]))  # density=False would make counts
  axs[cord].set_title(str(sp[0]+":"+str(reviewCountForTitle)),fontsize=108)
  # axs[cord].ylabel('Frequency')
  # axs[cord].xlabel('Data');
plt.tight_layout()
  

4 5
